# A1
* GCR = 0.25
* Albedo = 0.2
* Hub Height = 1.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* For a GCR of 0.25 the pitch is 9.536m
* 1-up Portrait: 5 rows, each with 25 modules
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/A1'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/A1


In [22]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv
from pathlib import Path

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Windows 10
Python version  3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.0
bifacial_radiance version  0.4.2+238.g3f0d94f.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [11]:
simulationname = 'A1'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1.303 # m
module_y = 2.384 # m. slope we will measure
sensorsy=2
torquetube_diam = 0.15

zgap = 0.02 # m
xgap = 0.002 # m

# SceneDict Parameters
pitch = 9.536 # m
albedo = 0.2
hub_height = 1.5 # m  
nMods = 25 
nRows = 5

sceneDict = {'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [6]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/A1
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [7]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.25,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [8]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y, xgap=xgap, zgap=zgap)


Module Name: PVmod
Module PVmod updated in module.json


In [12]:
module.addTorquetube(diameter=torquetube_diam, tubetype='Round', material='Metal_Grey', 
                     axisofrotation=True, visible=True, recompile=True)

Module PVmod updated in module.json
Pre-existing .rad file objects\PVmod.rad will be overwritten



In [32]:
weatherfile = str(Path().resolve().parent.parent / 'Instructions' / 'Phase2_meteo_hourly_psm3format.csv')
weatherfile

'C:\\Users\\sayala\\Documents\\GitHub\\bifacial_radiance\\docs\\development\\Instructions\\Phase2_meteo_hourly_psm3format.csv'

In [33]:
metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-01-01_1000', endtime='2022-01-01_1200')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [34]:
trackerdict = demo.set1axis(**trackerParams)

In [35]:
demo.gendaylit1axis()

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/


{'2022-01-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 29.16,
  'theta': -29.16,
  'dni': 5,
  'ghi': 112,
  'dhi': 108,
  'temp_air': 3.9,
  'wind_speed': 4.7,
  'skyfile': 'skies\\sky2_35.05_-106.54_2022-01-01_1030.rad'},
 '2022-01-01_1130': {'surf_azm': 90.0,
  'surf_tilt': 4.27,
  'theta': -4.27,
  'dni': 60,
  'ghi': 265,
  'dhi': 232,
  'temp_air': 3.3,
  'wind_speed': 5.5,
  'skyfile': 'skies\\sky2_35.05_-106.54_2022-01-01_1130.rad'}}

In [36]:
trackerdict = demo.makeScene1axis(module=module, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/

Making 2 octfiles in root directory.
Created 1axis_2022-01-01_1030.oct
Created 1axis_2022-01-01_1130.oct


In [37]:
trakerdict = demo.analysis1axis(sensorsy=2)

Linescan in process: 1axis_2022-01-01_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-01-01_1030_Row3_Module13_Back
Saved: results\irr_1axis_2022-01-01_1030_Row3_Module13.csv
Index: 2022-01-01_1030. Wm2Front: 101.55509999999998. Wm2Back: 13.994150000000001
Linescan in process: 1axis_2022-01-01_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-01-01_1130_Row3_Module13_Back
Saved: results\irr_1axis_2022-01-01_1130_Row3_Module13.csv
Index: 2022-01-01_1130. Wm2Front: 260.5147666666667. Wm2Back: 27.731835


In [38]:
trakerdict = demo.analysis1axis(sensorsy=2, modWanted = 1)

Linescan in process: 1axis_2022-01-01_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-01-01_1030_Row3_Module1_Back
Saved: results\irr_1axis_2022-01-01_1030_Row3_Module1.csv
Index: 2022-01-01_1030. Wm2Front: 102.25083333333333. Wm2Back: 17.829269999999998
Linescan in process: 1axis_2022-01-01_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-01-01_1130_Row3_Module1_Back
Saved: results\irr_1axis_2022-01-01_1130_Row3_Module1.csv
Index: 2022-01-01_1130. Wm2Front: 261.0045. Wm2Back: 41.920135


In [ ]:
trakerdict = demo.analysis1axis(sensorsy=2, modWanted = 25)

In [39]:
demo.calculateResults()

Bifaciality factor of module stored is  1
No CECModule data passed; using default for Prism Solar BHC72-400


c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

{'2022-01-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 29.16,
  'theta': -29.16,
  'dni': 5,
  'ghi': 112,
  'dhi': 108,
  'temp_air': 3.9,
  'wind_speed': 4.7,
  'skyfile': 'skies\\sky2_35.05_-106.54_2022-01-01_1030.rad',
  'radfile': 'objects\\1axis2022-01-01_1030__C_0.96549_rtr_9.53600_tilt_29.16000_25modsx5rows_origin0,0.rad',
  'scene': {'module': {'x': 1.303, 'y': 2.384, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.305, 'sceney': 2.384, 'scenez': 0.095, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmod 1.303 2.384 0.02 | xform -t -0.6515 -1.192 0.095 -a 1 -t 0 2.384 0\r\n! genrev Metal_Grey tube1 t*1.305 0.075 32 | xform -ry 90 -t -0.6525 0 0', 'modulefile': 'objects\\PVmod.rad', 'glass': False, 'offsetfromaxis': 0.095, 'xgap': 0.002, 'ygap': 0.0, 'zgap': 0.02}, 'modulefile': 'objects\\PVmod.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.25, 'text': '!xform -rx 29.16 -t 0 0 1.5 -a 25 -t 1.305 0 0 -a 5 -t 0 9.536 0 -i 1 -t -15.66 -19.072 0 -rz 90.0 -t 0 0 0 objects\\PVm

In [40]:
demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

,timestamp,module,row,POA_eff,Grear_mean,Gfront_mean,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI
0,2022-01-01_1030,13,3,115.549250,13.994150,101.555100,48.002683,42.060146,13.779859,14.128664,0.001051,48.002178,4.7,5,108,112
1,2022-01-01_1030,1,3,120.080103,17.829270,102.250833,49.927684,42.355437,17.436797,17.877863,0.000898,49.927236,4.7,5,108,112
2,2022-01-01_1130,13,3,288.246602,27.731835,260.514767,121.444050,109.756499,10.645015,10.648619,0.000030,121.444013,5.5,60,232,265
3,2022-01-01_1130,1,3,302.924635,41.920135,261.004500,127.611258,109.963281,16.061077,16.048973,0.000027,127.611223,5.5,60,232,265
